# 网络爬取图片

In [38]:
import urllib.request
import re
import requests
import time
import logging

LocalDay = time.strftime("%Y-%m-%d")
logFile = r'D:\python\{0}.log'.format(LocalDay)
logging.basicConfig(level=logging.DEBUG,
                 format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                 datefmt='%a, %d %b %Y %H:%M:%S',
                 filename=logFile,
                 filemode='a+')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

def getDatas(keyword, pages):
    params = []
    for i in range(30, 30 * pages + 30, 30):
        params.append({
            'tn': 'resultjson_com',
            'ipn': 'rj',
            'ct': 201326592,
            'is': '',
            'fp': 'result',
            'queryWord': keyword,
            'cl': 2,
            'lm': -1,
            'ie': 'utf-8',
            'oe': 'utf-8',
            'adpicid': '',
            'st': -1,
            'z': '',
            'ic': 0,
            'word': keyword,
            's': '',
            'se': '',
            'tab': '',
            'width': '',
            'height': '',
            'face': 0,
            'istype': 2,
            'qc': '',
            'nc': 1,
            'fr': '',
            'pn': i,
            'rn': 30,
            'gsm': '1e',
            '1526377465547': ''
        })
    url = 'https://image.baidu.com/search/index'
    urls = []
    for i in params:
        urls.append(requests.get(url, params=i).json().get('data'))
    return urls


def getImg(datalist, path):
    x = 0
    for list in datalist:
        for i in list:
            if i.get('thumbURL') != None:
                logging.info('正在下载：%s' % i.get('thumbURL'))
                urllib.request.urlretrieve(i.get('thumbURL'), path + '%d.jpg' % x)
                x += 1
            else:
                logging.info('当前轮次已经下载完成')

if __name__ == '__main__':
    queryKeyword = "猫狗"          #查询关键字
    pages = 10                     #下载页数
    totalPics = pages*30
    logging.info('开始下载%s的图片,预计下载图片%d张'%(queryKeyword,totalPics))
    datalist = getDatas(queryKeyword, pages)
    
    savepath="D://python//猫狗//"  #保存图片路径！
    if not os.path.exists(savepath):
        os.makedirs(savepath)
        
    getImg(datalist, os.path.join(savepath, 'img'))  #图片名字
 

root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     开始下载猫狗的图片,预计下载图片300张
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=1646918063,2048172889&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=1646918063,2048172889&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=1646918063,2048172889&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvH

root        : INFO     正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=3321822817,2200332709&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=3321822817,2200332709&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=3321822817,2200332709&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2796360382,2661687647&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2796360382,2661687647&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2796360382,2661687647&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2796360382,2661687647&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2796360382,2661687647&fm=26&gp=0.jpg
root        : IN

root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1697773357,2465042542&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3015724187,1649992690&fm=26&gp=0.jpg
root        : IN

root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=3793604818,1613435618&fm=26&gp=0.jpg
root        : IN

root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=413094177,3462410538&fm=26&gp=0.jpg
root        : INFO     正

...

root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=146790031,2307275443&fm=26&gp=0.jpg
root        : INFO     正

root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=3483765962,3089364890&fm=26&gp=0.jpg
root        : IN

root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1862521851,863467268&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=370615689,2115473000&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=370615689,2115473000&fm=26&gp=0.jpg
root        : INFO     正

root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=252311282,182722956&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=252311282,182722956&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=252311282,182722956&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=232976924,1144645047&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=232976924,1144645047&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=232976924,1144645047&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=232976924,1144645047&fm=11&gp=0.jpg
root        : INFO     正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=232976924,1144645047&fm=11&gp=0.jpg
root        : INFO     正在下载

root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : INFO     正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1424696105,3740905105&fm=26&gp=0.jpg
root        : IN